In [2]:
from ultralytics import YOLO
import cv2
import tkinter as tk
from tkinter import messagebox
import threading
import pyaudio
import wave
from pathlib import Path

In [3]:
# Checkin all classes for each model

# Load model YOLOv5n pre-trained with 80 labels
model = YOLO("yolov5nu.pt")

# Show labels
for i, label in model.names.items():
    print(f"{i}: {label}")

0: person
1: bicycle
2: car
3: motorcycle
4: airplane
5: bus
6: train
7: truck
8: boat
9: traffic light
10: fire hydrant
11: stop sign
12: parking meter
13: bench
14: bird
15: cat
16: dog
17: horse
18: sheep
19: cow
20: elephant
21: bear
22: zebra
23: giraffe
24: backpack
25: umbrella
26: handbag
27: tie
28: suitcase
29: frisbee
30: skis
31: snowboard
32: sports ball
33: kite
34: baseball bat
35: baseball glove
36: skateboard
37: surfboard
38: tennis racket
39: bottle
40: wine glass
41: cup
42: fork
43: knife
44: spoon
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
51: carrot
52: hot dog
53: pizza
54: donut
55: cake
56: chair
57: couch
58: potted plant
59: bed
60: dining table
61: toilet
62: tv
63: laptop
64: mouse
65: remote
66: keyboard
67: cell phone
68: microwave
69: oven
70: toaster
71: sink
72: refrigerator
73: book
74: clock
75: vase
76: scissors
77: teddy bear
78: hair drier
79: toothbrush


In [4]:
# Checkin all classes for each model

# Load model YOLOv5n pre-trained with 81 labels
model = YOLO("best.pt")

# Show labels
for i, label in model.names.items():
    print(f"{i}: {label}")

0: person
1: bicycle
2: car
3: motorcycle
4: airplane
5: bus
6: train
7: truck
8: boat
9: traffic light
10: fire hydrant
11: stop sign
12: parking meter
13: bench
14: bird
15: cat
16: dog
17: horse
18: sheep
19: cow
20: elephant
21: bear
22: zebra
23: giraffe
24: backpack
25: umbrella
26: handbag
27: tie
28: suitcase
29: frisbee
30: skis
31: snowboard
32: sports ball
33: kite
34: baseball bat
35: baseball glove
36: skateboard
37: surfboard
38: tennis racket
39: bottle
40: wine glass
41: cup
42: fork
43: knife
44: spoon
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
51: carrot
52: hot dog
53: pizza
54: donut
55: cake
56: chair
57: couch
58: potted plant
59: bed
60: dining table
61: toilet
62: tv
63: laptop
64: mouse
65: remote
66: keyboard
67: cell phone
68: microwave
69: oven
70: toaster
71: sink
72: refrigerator
73: book
74: clock
75: vase
76: scissors
77: teddy bear
78: hair drier
79: toothbrush
80: kids


In [ ]:
# Load model pre-trained to detect objects
import torch

model = YOLO("yolov5n.pt")
#model = YOLO("best.pt")

audio = 'beep.wav'

video = "video1.mp4" # choose the video name here to test more videos

# Load video
cap = cv2.VideoCapture(video)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Parameters
threshold_frames = 3
confidence_threshold = 0.45
consecutive_frames = 0
keep_beeping = False
alert_active = False  # NEW: flag to avoid multiple alerts

# Target labels to detect
target_labels = ['person', 'kids', 'cat', 'dog', 'horse', 'sheep', 'cow']

# Function: play alarm sound in loop
def beep_loop():
    global keep_beeping
    chunk = 1024

    wf = wave.open(audio, 'rb')
    p = pyaudio.PyAudio()

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    while keep_beeping:
        wf.rewind()
        data = wf.readframes(chunk)
        while data and keep_beeping:
            stream.write(data)
            data = wf.readframes(chunk)

    stream.stop_stream()
    stream.close()
    p.terminate()

# Function: show popup alert with sound
def show_alert(label, frames_detected):
    global keep_beeping, alert_active
    keep_beeping = True

    # Start beep in thread
    threading.Thread(target=beep_loop, daemon=True).start()

    # Show popup
    root = tk.Tk()
    root.withdraw()
    print(f"ALERT: {label.upper()} detected for {frames_detected} consecutive frames!")
    messagebox.showwarning(
        "DETECTION ALERT",
        f"ALERT: {label.upper()} detected for {frames_detected} consecutive frames!"
    )

    # Stop beep when popup closed
    keep_beeping = False
    root.destroy()

    # Reset alert flag
    alert_active = False

def center_window(window_name, frame_width, frame_height):
    # Get size of the screen
    root = tk.Tk()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()

    # Calculating the position to center the window
    x = int((screen_width - frame_width) / 2)
    y = int((screen_height - frame_height) / 2)

    # Moveing the window to the center
    cv2.moveWindow(window_name, x, y)

window_name = "Detection"

# Create window with normal size and center it
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
center_window(window_name, frame_width, frame_height)

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model(frame, verbose=False)
    annotated = results[0].plot()

    # Check for target labels
    detected = False
    detected_label = ""

    for box in results[0].boxes:
        conf = box.conf.item()
        label = results[0].names[int(box.cls)]

        if label.lower() in target_labels and conf > confidence_threshold:
            detected = True
            detected_label = label
            break

    # Update counter
    consecutive_frames = consecutive_frames + 1 if detected else 0

    # Trigger alert (if not already active)
    if consecutive_frames >= threshold_frames and not alert_active:
        alert_active = True
        threading.Thread(target=show_alert, args=(detected_label, consecutive_frames), daemon=True).start()
        consecutive_frames = 0

    # Show video with detections
    cv2.imshow("Detection", annotated)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# press "q" to fisnih the videoq


ALERT: KIDS detected for 4 consecutive frames!
ALERT: KIDS detected for 5 consecutive frames!
ALERT: KIDS detected for 5 consecutive frames!
